In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import trange

import random
import math
from abc import ABC, abstractmethod
import os
import numpy as np
from numpy.ctypeslib import ctypes as ctypes
from numpy.ctypeslib import ctypes as ctypes
from abc import ABC, abstractmethod
import os
import numpy as np

EBoardP =ctypes.c_void_p
argType=ctypes.c_int
actionT=ctypes.c_uint16
pointer_action=ctypes.POINTER(actionT)

# FIXME: quando vedi i figli salvali


# Abstract class that implements the game rules
class game_rule(ABC):
    @abstractmethod
    def init_state(self):
        pass
    def next_state(self, state, action:actionT):
        pass
    def checkStatus(self, state):
        pass

# Our game's rules
class DLLGameRule(game_rule):

    MAX_ACTIONS = 1575
    BOARD_VECTOR_SIZE = 32*32

    def __init__(self):
        path = os.path.join(os.getcwd(), 'cpp/interface.so')
        dll = ctypes.CDLL(path)
        self.dll=dll
        self.getBoard_low = dll._Z10base_statei
        self.copyBoard = dll._Z9copyBoardP6EBoard
        self.updateState_low = dll._Z10next_stateP6EBoardt
        self.checkGameStatus_low = dll._Z11checkStatusP6EBoard
        #self.printAct_low = dll._Z16printActionFancym
        self.PrintBoard_low = dll._Z10printBoardP6EBoard
        self.delBoardC_low=dll._Z8delBoardP6EBoard
        self.stringToAction_low=dll._Z14stringToActionP6EBoardPc
        #self.boardToCVect_low=dll._Z9BoardRappP6EBoard
        self.getStatusVector_low=dll._Z15getStatusVectorP6EBoard
        self.getAssociatedAction_low=dll._Z19getAssociatedActionP6EBoard
        self.getMask_low=self.dll._Z7getMaskP6EBoard
        self.compute_possible_low=self.dll._Z19updatePossiblemovesP6EBoard;
        # Set argument/return types
        self.getBoard_low.argtypes = [argType]  # the type of game, define in engine/enums.h
        self.getBoard_low.restype = EBoardP

        self.copyBoard.argtypes = [EBoardP]
        self.copyBoard.restype = EBoardP

        self.updateState_low.argtypes = [EBoardP, actionT]
        self.updateState_low.restype = None

        self.checkGameStatus_low.argtypes = [EBoardP]
        self.checkGameStatus_low.restype = ctypes.c_int

        self.stringToAction_low.argtypes=[EBoardP,ctypes.c_char_p]
        self.stringToAction_low.restype=actionT

        self.actions = (actionT * self.MAX_ACTIONS)()

        self.getMask_low.argtypes = [ctypes.c_void_p]
        self.getMask_low.restype = ctypes.POINTER(ctypes.c_uint8)

        self.getAssociatedAction_low.argtypes = [ctypes.c_void_p]
        self.getAssociatedAction_low.restype = ctypes.POINTER(actionT)

        self.getStatusVector_low.argtypes = [ctypes.c_void_p]
        self.getStatusVector_low.restype = ctypes.POINTER(actionT)

        self.strBuff=(ctypes.c_char* 30)()

    # Returns the initial state
    def init_state(self,gametype=7) -> EBoardP:
        return self.getBoard_low(gametype)  # default game type  0 ->base, ..., 7 -> MPL


    # Update state given an action. action is a number. Doesn't return anything
    def next_state(self, state:EBoardP, action:actionT): # FIXME: fa l'update della board che ci passi (action come intero)
        self.updateState_low(state, actionT(action))

    # Get the set of possible actions. It is a vector of actions (ints).
    STATUS_OK = 0
    STATUS_INVALID = 1
    STATUS_WHITE_WINS = 2
    STATUS_BLACK_WINS = 3

    # Check if someone has won
    def checkStatus(self, state:EBoardP): # FIXME: ritorna il numero in base a cosa è successo
        code = self.checkGameStatus_low(state)
        status_map = {
            0: "OK",
            1: "INVALID_GAME_NOT_STARTED",
            2: "GAME_OVER_WHITE_WINS or DRAW",
            3: "GAME_OVER_BLACK_WINS",
            4: "GAME_OVER_DRAW"
        }
        return code



    # Prints the board
    def PrintState(self, state:EBoardP): # FIXME: for debug
        self.PrintBoard_low(state)

    # Deletes the board
    def delBoard(self,state:EBoardP):
        self.delBoardC_low(state)

    # Converts the board to a vector of chars (284)
    def toVect(self,state:EBoardP): # FIXME: sarebbe ottimo ritornare un numpy vector di 32*32 CNN 32*32 più altri neuroni (6 in teoria, +1 che rappresenta il turno (è endgame o startgame) +2 che sono i bug mossi appena adesso) che ci dicono chi è sopra a chi e cose del genere
        ptr = self.getStatusVector_low(state)
        arr = np.ctypeslib.as_array(ptr, shape=(1033,))
        return torch.from_numpy(arr.astype(np.float32))

    def toVect_split(self, state: EBoardP) -> tuple[torch.FloatTensor, torch.FloatTensor]:
        ptr = self.getStatusVector_low(state)
        arr = np.ctypeslib.as_array(ptr, shape=(1033,)).astype(np.float32)

        board = torch.from_numpy(arr[:1024]).view(1, 32, 32).to(device)
        metadata = torch.from_numpy(arr[1024:]).to(device)
        return board, metadata

    # Converts a string to an action
    def stringToAction(self,state:EBoardP,str): #FIXME: ritorna un intero
        return self.stringToAction_low(state,str.encode())

    def get_mask(self, state):          # mask for actions
        ptr = self.getMask_low(state)
        return np.ctypeslib.as_array(ptr, shape=(self.MAX_ACTIONS,)).astype(bool)

    def get_actions(self,state):   # all actions masked by getMask
        ptr = self.getAssociatedAction_low(state)
        return np.ctypeslib.as_array(ptr, shape=(self.MAX_ACTIONS,)).astype(np.uint64)

    def compute_actions(self,state:EBoardP):
        self.compute_possible_low(state)

    def get_opponent_value(self, value):
        return -value

    def get_opponent(self, player): # TODO: il player è 1 o -1
        return -player

In [4]:
def competition(agent1,agent2,roundNumber):
    win={agent1.name:0,agent2.name:0}
    for R in range(roundNumber):
        numR=random.randint(0,1);
        agent1.reset()
        agent2.reset()
        if numR==0:
            agents=[agent1,agent2]
        else:
            agents=[agent2,agent1]
        
        state=GR.init_state()
        agent1.reset()
        agent2.reset()
        t=0
        while True:
            t+=1

            M=agents[t%2].bestmove()
            agents[0].playmove(M)
            agents[1].playmove(M)
            GR.next_state(state,GR.stringToAction(state,M))
            print(M)
            if(GR.checkStatus(state)!=1):
                win[agents[(t%2)].name]+=1
                break
            print("Eval:",agents[1-numR].numEval)
        print(win)
        GR.delBoard(state)
    

In [5]:
aR=RandomAgentCPP()
aS=MinimaxAgentCPP(depth=6)




NameError: name 'RandomAgentCPP' is not defined

In [6]:
competition(aR,aS,100)

NameError: name 'aR' is not defined